# LIME

### Metrics tested are identity, fidelity, separability, speed.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import lime.lime_tabular
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import time
import tqdm
import metrics_rules
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Define the path to the datasets folder
datasets_folder = "datasets"

# Initialize empty lists to store dataframes for each file
folder_names = []
attribute_names_list = []
categorical_indicator_list = []
X_list = []
y_list = []

# Loop through each folder in the datasets folder
for folder_name in os.listdir(datasets_folder):
    folder_path = os.path.join(datasets_folder, folder_name)
    
    # Check if it's a directory
    if os.path.isdir(folder_path):
        # Construct file paths for each CSV file in the folder
        attribute_names_path = os.path.join(folder_path, "attribute_names.csv")
        categorical_indicator_path = os.path.join(folder_path, "categorical_indicator.csv")
        X_path = os.path.join(folder_path, "X.csv")
        y_path = os.path.join(folder_path, "y.csv")
        
        # Read each CSV file into a pandas dataframe
        attribute_names_df = pd.read_csv(attribute_names_path)
        categorical_indicator_df = pd.read_csv(categorical_indicator_path)
        X_df = pd.read_csv(X_path)
        y_df = pd.read_csv(y_path)
        
        # Append dataframes to the lists
        attribute_names_list.append(attribute_names_df)
        categorical_indicator_list.append(categorical_indicator_df)
        X_list.append(X_df)
        y_list.append(y_df)

        # Save folder name to list
        folder_names.append(folder_name)

# Subsetting for less expensive runs
X_list = [df.head(50) for df in X_list]
y_list = [df.head(50) for df in y_list]

# For testing the techniques
X = X_list[:40]
y = y_list[:40]

# Names of chosen datasets
X_folder_names = folder_names[:40]

# For testing later
X_list_test = X_list[-10:]
y_list_test = y_list[-10:]

In [ ]:
# Preprocessing
def convert_to_numeric_and_impute(X_list, y_list):
    imputer = SimpleImputer(strategy='mean')
    label_encoder = LabelEncoder()

    def process_dataframe(df):
        for column in df.columns:
            if isinstance(df[column].iloc[0], csr_matrix):
                df[column] = df[column].apply(lambda x: x.toarray()[0,0] if x.shape[1] == 1 else x.toarray())

            df[column] = pd.to_numeric(df[column], errors='coerce')

            if df[column].dtype == 'object':
                # Fill NaN with a placeholder and then label encode
                df[column] = df[column].fillna('Missing')
                df[column] = label_encoder.fit_transform(df[column])
            else:
                if df[column].notna().any():
                    df[column] = imputer.fit_transform(df[[column]]).ravel()
                else:
                    df[column] = df[column].fillna(0)

        return df

    X_list = [process_dataframe(df) for df in X_list]
    y_list = [process_dataframe(df) for df in y_list]

    return X_list, y_list

X, y = convert_to_numeric_and_impute(X, y)
X_list_test, y_list_test = convert_to_numeric_and_impute(X_list_test, y_list_test)

In [ ]:
def approximate_lime_predictions(lime_weights, perturbed_samples):
    lime_predictions = np.zeros(len(perturbed_samples))
    for i, sample in enumerate(perturbed_samples):
        prediction = 0
        for feature_index, weight in lime_weights:
            prediction += sample[feature_index] * weight
        lime_predictions[i] = prediction
    return lime_predictions

def apply_lime_model_to_test_set(X_test, explanation, num_classes):
    lime_weights = explanation.local_exp
    lime_model_predictions = np.zeros((X_test.shape[0], num_classes))

    for i in range(X_test.shape[0]):
        for class_idx in range(num_classes):
            class_weights = dict(lime_weights[class_idx])
            prediction = sum(X_test.iloc[i][feature] * weight for feature, weight in class_weights.items())
            lime_model_predictions[i, class_idx] = prediction

    return np.argmax(lime_model_predictions, axis=1)

def generate_perturbed_samples(instance, num_features, num_samples=100):
    perturbed_samples = []
    for _ in range(num_samples):
        perturbed_instance = instance.copy()
        for feature in range(num_features):
            perturbation = np.random.normal(0, 0.01)
            perturbed_instance[feature] += perturbation
        perturbed_samples.append(perturbed_instance)
    return np.array(perturbed_samples)

def approximate_lime_predictions(lime_weights, perturbed_samples, num_classes):
    lime_predictions = np.zeros((len(perturbed_samples), num_classes))
    for i, sample in enumerate(perturbed_samples):
        for class_idx in range(num_classes):
            if class_idx in lime_weights:
                class_weights = dict(lime_weights[class_idx])
                prediction = sum(sample[feature] * weight for feature, weight in class_weights.items())
                lime_predictions[i, class_idx] = prediction
    return lime_predictions

def calc_lime_model_accuracy(X_test, y_test, model, lime_explainer, num_classes, num_samples=100):
    lime_accuracies = []
    for index, instance in enumerate(X_test.values):
        try:
            explanation = lime_explainer.explain_instance(instance, model.predict_proba, num_features=len(X_test.columns))
            perturbed_samples = generate_perturbed_samples(instance, len(X_test.columns), num_samples)
            lime_weights = explanation.local_exp
            lime_predictions = approximate_lime_predictions(lime_weights, perturbed_samples, num_classes)
            predicted_classes = np.argmax(lime_predictions, axis=1)
            lime_accuracy = accuracy_score(y_test.iloc[index:index+1].repeat(num_samples), predicted_classes)
            lime_accuracies.append(lime_accuracy)
        except Exception as e:
            print(f"Error in LIME model accuracy calculation for instance {index}: {e}")
            lime_accuracies.append(None)
    return np.nanmean(lime_accuracies)

In [ ]:
df_interp = pd.DataFrame(columns=["Dataset", "Fidelity", "Identity", "Separability", "Speed"])

for i in range(len(X_list)):
    print(f"Dataset {i}")
    X, y = X_list[i], y_list[i].squeeze()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=555)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        training_data=X_train.values, 
        feature_names=X_train.columns.tolist(), 
        class_names=np.unique(y).tolist(), 
        discretize_continuous=True
    )

    # Define the function to create an explanation for one instance
    def exp_fn(i):
        instance = X_test.iloc[i].values
        return lime_explainer.explain_instance(instance, model.predict_proba, num_features=len(X_train.columns))

    # Define the function to apply the explanation function to a block of instances
    def exp_fn_blk(xtest):
        exp1 = []
        for i in tqdm.tqdm(range(len(xtest))):
            exp = exp_fn(i)
            exp1.append(exp.as_map()[exp.available_labels()[0]])
        return np.array(exp1)

    
    start_time = time.time() 
    exp1 = exp_fn_blk(X_test.values)
    exp2 = exp_fn_blk(X_test.values)
    end_time = time.time()
    speed = end_time - start_time 

    lime_accuracy = calc_lime_model_accuracy(X_test, y_test, model, lime_explainer, num_classes=2)

    # Add the results to the dataframe
    df_interp = df_interp.append({
        "Dataset": i,
        "Fidelity": lime_accuracy,  # Placeholder for Fidelity
        "Identity": metrics_rules.calc_identity_rules(exp1[0], exp2[0])[0],
        "Separability": metrics_rules.calc_separability_rules(exp1[0])[-1],
        "Speed": speed # Placeholder for Speed
    }, ignore_index=True)

print(df_interp)


In [ ]:
# Adding "lime_" prefix to every column name
df_interp.columns = ['lime_' + col for col in df_interp.columns]
df_interp

In [ ]:
df_interp.to_csv('records_lime.csv')